In [1]:
class NumBuff(list):
    def __init__(self, iterable=(),maxlen=128):
        self.maxlen=maxlen
        assert len(iterable)<=maxlen
        super().__init__(iterable)
        self.num=0
        #start_point_num
        
    def extend(self):
        raise NotImplementedError
    
    def keep_len(self):
        for _ in range(len(self)-self.maxlen):
            if len(self)>self.maxlen:
                self.pop(0)
                self.num+=1
    
    def append(self, item):
        self.keep_len()
        super().append(item)

In [2]:
class NumDecoder:
    def __init__(self):
        self.li=NumBuff(maxlen=12)
        #just for test
        self.old_li=[]
        
        self.waiting={}
        self.now_num=0
        #end_point_num
        
        self.get_len=10
        
    def to_get_len(self):
        i=self.waiting.items()
        if not len(i):return False
        i=i[0]
        self.get_len=len(i)-1
        return True
    
    def recycle(self):
        for key in list(self.waiting.keys()):
            if self.now_num-key>self.get_len*2:
                self.waiting.pop(key)
    
    def new_li(self,num):
        self.now_num=num
        self.old_li=self.li
        self.li=NumBuff()
        self.li.num=num
        self.waiting={}
    
    def __call__(self,num,*things):
        self.recycle()
        
        if len(self.li)==0:
            self.new_li(num)
        
        
        #must have looped
        if self.now_num-num>1024:
            self.new_li(num)
            #just, drop it.
            #think about it later
        
        self.waiting[num]=things
        self.transfer_waiting()
        
    @property
    def max_num(self):
        return len(self.li)+self.li.num
    
    def transfer_waiting(self):
        
        nums=self.waiting.keys()
        nums=[i for i in nums if i<=self.max_num]
        
        for num in nums:
            things=self.waiting[num]
            index=num-self.li.num
            if index<0:continue
            self.li[index:]=things
            self.waiting.pop(num)
            
        
        self.li.keep_len()
        
        self.now_num=self.max_num
      

In [3]:
C=NumDecoder()

In [4]:
C.li

[]